In [ ]:
!pip install tensorflow==2.3.0
!pip install tensorflow-addons==0.11.1

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.optimizers import Adam
import random

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

from sklearn.utils import class_weight as cw

In [ ]:
tf.__version__

In [ ]:
batch_size = 16
original_img_size = (256, 2000)
model_img_size = 256
seed = 1
storage_dir = '../input/birdmel/train_img_split/train_img_split'
AUTOTUNE = tf.data.experimental.AUTOTUNE
crop_height = model_img_size
crop_width = 216

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    storage_dir,
    validation_split=0.1,
    subset="training",
    seed=seed,
    image_size=original_img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    storage_dir,
    validation_split=0.1,
    subset="validation",
    seed=seed,
    image_size=original_img_size,
    batch_size=batch_size,
    label_mode='categorical',
)

In [ ]:
def build_model(num_classes, img_size=model_img_size):
    inputs = layers.Input(shape=(img_size, img_size, 3))
    model = EfficientNetB4(include_top=False, input_tensor=inputs)

    x = layers.GlobalMaxPooling2D(name="max_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.5
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    model.compile(
        optimizer=Adam(learning_rate=1e-3), 
        loss="categorical_crossentropy", 
        metrics=[tfa.metrics.F1Score(num_classes = class_num, average = 'micro')]
    )

    return model

In [ ]:
random_crop_layer = preprocessing.RandomCrop(crop_height, crop_width)
random_translate_layer = preprocessing.RandomTranslation(height_factor=0, width_factor=0.1, fill_mode='constant')
rescale_layer = preprocessing.Rescaling(scale=1./255)
random_contrast_layer = preprocessing.RandomContrast(factor=0.1)
gaussian_noise_layer = layers.GaussianNoise(0.1)

def resize(image, height=model_img_size, width=model_img_size):
    return tf.image.resize_with_crop_or_pad(image, target_height=height, target_width=width)

def mask(image, mask_length=10):
    insertion_index = int(model_img_size/2)
    mask_start = int(mask_length/2)

    mask_end_time = crop_width - mask_start + 1
    mask_offset_time = random.randrange(mask_start, mask_end_time)

    mask_end_freq = crop_height - mask_start + 1
    mask_offset_freq = random.randrange(mask_start, mask_end_freq)
    
    image = tfa.image.cutout(
        image,
        mask_size=(model_img_size, mask_length),
        offset=(insertion_index, mask_offset_time)
    )
    
    image = tfa.image.cutout(
        image,
        mask_size=(mask_length, model_img_size),
        offset=(mask_offset_freq, insertion_index)
    )
    
    return image

def augment_image_train(image):
    image = random_crop_layer(image)
    image = random_contrast_layer(image)
    image = mask(image)
    image = random_translate_layer(image)
    image = gaussian_noise_layer(image, training=True)
    
    image = resize(image)
    image = rescale_layer(image)
    
    return image

def augment_image_test(image):
    image = resize(image, crop_height, crop_width)

    image = resize(image)
    image = rescale_layer(image)
    
    return image

In [ ]:
def prepare(ds, augment=False):
    # Use data augmentation only on the training set
    if augment:
        ds = ds.map(lambda x, y: (augment_image_train(x), y), num_parallel_calls=AUTOTUNE)
    else: 
        ds = ds.map(lambda x, y: (augment_image_test(x), y), num_parallel_calls=AUTOTUNE)

    # Use buffered prefecting on all datasets
    return ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
class_num = len(train_ds.class_names)

In [ ]:
train_ds = prepare(train_ds, augment=True)
val_ds = prepare(val_ds)

In [ ]:
train_datagen = ImageDataGenerator(dtype=np.float16)

train_generator = train_datagen.flow_from_directory(
    directory=storage_dir,
    class_mode="categorical",
    target_size=original_img_size,
    batch_size=batch_size
)

class_weights = cw.compute_class_weight(
    'balanced',
    np.unique(train_generator.classes), 
    train_generator.classes
)

class_weights = {i : class_weights[i] for i in range(len(class_weights))}

del train_datagen
del train_generator

In [ ]:
net = build_model(class_num, model_img_size)

In [ ]:
net.load_weights( '../input/birdmel/efficientnet_b4_checkpoint_tf_2_3_0.h5')

In [ ]:
model_check = ModelCheckpoint(
    'efficientnet_b4_checkpoint_tf_2_3_0.h5', 
    monitor='val_f1_score', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=True,
    mode='max',
    period=1,
)


reduce_LR = ReduceLROnPlateau(
    monitor='val_f1_score',
    factor=0.2,
    patience=5, 
    min_lr=5e-5
)

In [ ]:
net.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 100,
    class_weight = class_weights,
    callbacks = [reduce_LR, model_check]
)